# Step - 1 Load the data

1a - Loaded into the drive from github

1b - Merge the Datasets using product_locale and product_id (connects each query/label pair to its corresponding product information)

1c - Filtering the large merged DataFrame down to the required data




In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df_examples =pd.read_parquet('/content/drive/MyDrive/Grainger_Ashrith/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('/content/drive/MyDrive/Grainger_Ashrith/shopping_queries_dataset_products.parquet')


In [ ]:
df_examples.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train


In [ ]:
df_products.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es


In [ ]:
df_merged = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

In [ ]:
# filter for Task 1 (using the 'small_version' flag)
df_task_1 = df_merged[df_merged["small_version"] == 1]

# filter for the training set
df_task_1_train = df_task_1[df_task_1["split"] == "train"]

# applying filters for 'us' locale and 'E' label
final_training_data = df_task_1_train[
    (df_task_1_train["product_locale"] == "us") &
    (df_task_1_train["esci_label"] == "E")
]

In [ ]:
# final_training_data
final_training_data.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver
21,21,!awnmower tires without rims,1,B077QMNXTS,us,E,1,1,train,MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Ga...,<br>Tire Specifications:<br> 1. Material: Rubb...,"Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn ...",MaxAuto,Black
23,23,!awnmower tires without rims,1,B06XX6BM2R,us,E,1,1,train,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assem...",None,Tire: 2 pack 15x6. 00-6 tube-type turf SAVER t...,MARASTAR,None
26,26,!awnmower tires without rims,1,B0089RNSNM,us,E,1,1,train,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Set of 2 Honda OEM Rear Wheels\nReplaces 42710...,Honda,None


In [ ]:
final_training_data_copy = final_training_data.copy(deep=True)
print("Created a copy of final_training_data named final_training_data_copy.")

# Step - 2: Create sample dataset

First we create a 500-row sample from the training data, centered on 50 unique queries.

2a: random sampling of rows

2b: (Simple Random Sampling), select 50 unique queries and then sample row

2c: Stratified sampling with around 60 unique queries to find 50 that yield a sample close to 500 rows

In [ ]:
df_full = final_training_data.reset_index(drop=True)
print(f"1. Total available rows in master dataset: {len(df_full)}")

1. Total available rows in master dataset: 181819


#### 2a: Basic barebones


In [ ]:
# --- Configuration ---
RANDOM_STATE = 42
TARGET_QUERIES = 50 # This target will be used in later steps
TARGET_ROWS = 500
# ---------------------

# Assuming final_training_data is a pre-loaded pandas DataFrame
# This is the master dataset (Task 1, train, 'us', 'E' label, etc.)
df_full = final_training_data.reset_index(drop=True)
print(f"1. Total available rows in master dataset: {len(df_full)}")

print(f"\n--- Step 2a: Simple Random Sample ---")

# Take a simple random sample of TARGET_ROWS from the full dataset
if len(df_full) <= TARGET_ROWS:
    sample_2a = df_full.copy()
    print(f"Full dataset is too small ({len(df_full)} rows), using all available rows.")
else:
    sample_2a = df_full.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"We took a simple random sample of {len(sample_2a)} rows from the full dataset.")

final_unique_2a = sample_2a['query'].nunique()

print(f"Result: {final_unique_2a} unique queries in the sample.")

print("\nSample 2a Head:")
display(sample_2a[['query', 'product_title']].head())

1. Total available rows in master dataset: 181819

--- Step 2a: Simple Random Sample ---
We took a simple random sample of 500 rows from the full dataset.
Result: 490 unique queries in the sample.

Sample 2a Head:


,query,product_title
0,drainer for sink,Outivity Sink Drain Shelf Triangular Sink Bask...
1,chernobyl,Azure Dust: Inside Chernobyl's Exclusion Zone
2,retaine eye drops for dry eyes preservative free,UrsaPharm Hylo-Forte Lubricating Eye Drops 10M...
3,long sleeve pink dress,R.Vivimos Women's Autumn Winter Cotton Long Sl...
4,(hearing aid not amplifer),"Hearing Aids, Enjoyee Hearing Aids for Seniors..."


#### Step 2b: Simple Random Sample

We just randomly grab 500 rows. This can drop queries with few associated products.

In [ ]:
print(f"\n--- Step 2b: Sample from Filtered Queries ---")

# 1. Randomly select 50 unique queries
unique_queries_2b = df_full['query'].unique()
if len(unique_queries_2b) < TARGET_QUERIES:
    sample_queries_2b = pd.Series(unique_queries_2b)
else:
    # Use the predefined RANDOM_STATE for reproducibility
    sample_queries_2b = pd.Series(unique_queries_2b).sample(n=TARGET_QUERIES, random_state=RANDOM_STATE)

# 2. Filter the master set down to only the products matching these 50 queries
df_filtered_2b = df_full[df_full['query'].isin(sample_queries_2b)].reset_index(drop=True)
total_rows_for_queries_2b = len(df_filtered_2b)

print(f"-> We selected {len(sample_queries_2b)} unique queries.")
print(f"-> Total associated rows available after filtering: {total_rows_for_queries_2b}")

# 3. Take a sample of 500 rows from the filtered dataset
if total_rows_for_queries_2b <= TARGET_ROWS:
    sample_2b = df_filtered_2b.copy()
    print(f"Filtered set is too small ({total_rows_for_queries_2b} rows), using all available rows.")
else:
    # Take a random sample of TARGET_ROWS from the filtered dataset using the predefined RANDOM_STATE
    sample_2b = df_filtered_2b.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"We took a random sample of {len(sample_2b)} rows from the filtered dataset.")


# 6. Print the number of rows and unique queries in the resulting sample.
final_unique_2b = sample_2b['query'].nunique()
print(f"Result: {len(sample_2b)} rows and {final_unique_2b} unique queries in the sample.")

# 7. Display the head of the resulting sample DataFrame
print("\nSample 2b Head:")
display(sample_2b[['query', 'product_title']].head())


--- Step 2b: Sample from Filtered Queries ---
-> We selected 50 unique queries.
-> Total associated rows available after filtering: 329
Filtered set is too small (329 rows), using all available rows.
Result: 329 rows and 50 unique queries in the sample.

Sample 2b Head:


,query,product_title
0,6 dining chairs,Yaheetech Dining Chairs Velvet Armchairs for C...
1,6 dining chairs,CozyCasa Dining Chairs Modern Style Dining Cha...
2,6 dining chairs,Yaheetech Dining Chairs with Waterproof leathe...
3,6 dining chairs,Yaheetech Dining Chairs Dining Room Chairs Liv...
4,6 dining chairs,Modern Dining Chairs Set of 6 - Faux Leather D...


#### 2c Stratified sampling

We ensure one row per query is selected first, then fill the rest.

In [ ]:
print(f"\n--- Step 2c: Stratified Sampling for Target Rows and Queries ---")

# Configuration for Step 2c
TARGET_QUERIES_2c = 70 # Aim for a higher number of unique queries initially

# 1. Randomly select a slightly higher number of unique queries (e.g.,70)
unique_queries_full = df_full['query'].unique()
if len(unique_queries_full) < TARGET_QUERIES_2c:
    potential_sample_queries = pd.Series(unique_queries_full)
    print(f"Only {len(unique_queries_full)} unique queries available in the full dataset, using all.")
else:
    potential_sample_queries = pd.Series(unique_queries_full).sample(n=TARGET_QUERIES_2c, random_state=RANDOM_STATE)
    print(f"Randomly selected {len(potential_sample_queries)} potential unique queries.")

# 2. Filter the df_full DataFrame to include only rows associated with the selected queries.
df_potential_queries = df_full[df_full['query'].isin(potential_sample_queries)].reset_index(drop=True)
print(f"Filtered down to {len(df_potential_queries)} rows associated with these potential queries.")


# 3. Calculate the number of rows per query in df_potential_queries.
query_row_counts = df_potential_queries.groupby('query').size().sort_values(ascending=False)
print("\nRow counts for potential queries:")
display(query_row_counts.head())

# 4. Determine which 50 queries get closest to TARGET_ROWS (500)
# Sort queries by row count and greedily select the top 50 that get closest to 500
selected_queries_2c = []
current_row_count = 0
for query, count in query_row_counts.items():
    if len(selected_queries_2c) < 50: # Select up to 50 queries
         selected_queries_2c.append(query)
         current_row_count += count
    else:
        # Once we have 50 queries, we can stop or refine the selection
        # A simple greedy approach is to just take the top 50 by count
        # If more sophisticated selection needed (e.g., closest to 500), it would go here
        pass

# Ensure we have exactly 50 queries if enough were available
if len(selected_queries_2c) < 50 and len(potential_sample_queries) >= 50:
     print(f"Warning: Could only select {len(selected_queries_2c)} queries as not enough queries were available.")
elif len(selected_queries_2c) == 50:
     print(f"\nSelected 50 queries that yield a total of {current_row_count} rows.")


# 5. Filter df_potential_queries to keep only the rows corresponding to these final 50 queries.
df_final_50_queries = df_potential_queries[df_potential_queries['query'].isin(selected_queries_2c)].reset_index(drop=True)

# 6. Determine the final sample based on the number of rows
if len(df_final_50_queries) == TARGET_ROWS:
    sample_2c = df_final_50_queries.copy()
    print(f"Final filtered set has exactly {TARGET_ROWS} rows.")
elif len(df_final_50_queries) < TARGET_ROWS:
    sample_2c = df_final_50_queries.copy()
    print(f"Final filtered set has {len(df_final_50_queries)} rows, less than the target {TARGET_ROWS}. Using all available rows.")
else: # len(df_final_50_queries) > TARGET_ROWS
    sample_2c = df_final_50_queries.sample(n=TARGET_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"Final filtered set has {len(df_final_50_queries)} rows, more than the target {TARGET_ROWS}. Taking a random sample of {TARGET_ROWS}.")


# 7. Print the final number of rows and unique queries in the sample_2c DataFrame.
final_unique_2c = sample_2c['query'].nunique()
print(f"\nFinal Sample (2c) Result: {len(sample_2c)} rows and {final_unique_2c} unique queries.")

# 8. Print the head of the sample_2c DataFrame.
print("\nFinal Sample (2c) Head:")
display(sample_2c[['query', 'product_title', 'esci_label']].head())


--- Step 2c: Stratified Sampling for Target Rows and Queries ---
Randomly selected 70 potential unique queries.
Filtered down to 520 rows associated with these potential queries.

Row counts for potential queries:


,0
query,
turning shoe,30
6 dining chairs,22
plants,22
tan and brown bathroom wall decor,21
poleras deportivas mujer,19



Selected 50 queries that yield a total of 485 rows.
Final filtered set has 485 rows, less than the target 500. Using all available rows.

Final Sample (2c) Result: 485 rows and 50 unique queries.

Final Sample (2c) Head:


,query,product_title,esci_label
0,6 dining chairs,Yaheetech Dining Chairs Velvet Armchairs for C...,E
1,6 dining chairs,CozyCasa Dining Chairs Modern Style Dining Cha...,E
2,6 dining chairs,Yaheetech Dining Chairs with Waterproof leathe...,E
3,6 dining chairs,Yaheetech Dining Chairs Dining Room Chairs Liv...,E
4,6 dining chairs,Modern Dining Chairs Set of 6 - Faux Leather D...,E


In [ ]:
# Save the sample_2c DataFrame to a CSV file
sample_2c.to_csv('/content/drive/MyDrive/Grainger_Ashrith/Final/sample_2c_full_data.csv', index=False)

Saved the full sample_2c DataFrame to /content/drive/MyDrive/Grainger_Ashrith/Final/sample_2c_full_data.csv


In [ ]:
sample_2c.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,118060,6 dining chairs,4845,B08CZ6TC2L,us,E,1,1,train,Yaheetech Dining Chairs Velvet Armchairs for C...,Set of 6 Kitchen Dining Chairs for Counter Lou...,STRONG METAL LEGS: To enhance the weight capac...,Yaheetech,Grey
1,118064,6 dining chairs,4845,B08HQG1MFS,us,E,1,1,train,CozyCasa Dining Chairs Modern Style Dining Cha...,<b>If you are in search of some quality-reliab...,Dining Chairs set of 6 -- White PP backrest an...,CozyCasa,White
2,118065,6 dining chairs,4845,B08K2K3J4C,us,E,1,1,train,Yaheetech Dining Chairs with Waterproof leathe...,Make every long-time sitting comfortable. The ...,MULTIPLE USE: Sold in a set of 6 chairs. Desig...,Yaheetech,Brown
3,118066,6 dining chairs,4845,B08K2V66N8,us,E,1,1,train,Yaheetech Dining Chairs Dining Room Chairs Liv...,Make every dinner time comfortable. Constructe...,MULTIPLE USE: Sold in a set of 6 chairs. This ...,Yaheetech,Khaki
4,118067,6 dining chairs,4845,B08K8VDTW8,us,E,1,1,train,Modern Dining Chairs Set of 6 - Faux Leather D...,<b>Modern Dining Chairs Set of 6 - Faux Leathe...,Comfortable Dining Chairs Set of 6 - The dinin...,WENYU,Grey


### Some Data Explroing

Intresting queires: One query was  "usb2aub2ra1m"

And its apprently a product id for right anlged usb connector

https://www.startech.com/en-eu/cables/usb2aub2ra1m?srsltid=AfmBOoryvB93OxhVQnPUAocknMNz41MVDvr2TJMrWf0ijRnCwf5htlXn

Face urine? - Fake urine but still never knew these existed haha

And some plumbing related queires: zurn qkipsp 5 port plastic manifold without valves

# Step 3: Vector Index

3a - Baseline tf-idf

3b - Dense model (all-MiniLM-L6-v2)

3c - Two tower dense model

3d - Hybrid model



REfrence links:

Uber blog on two tower arch: https://www.uber.com/blog/innovative-recommendation-applications-using-two-tower-embeddings/


https://www.kaggle.com/code/abhishekmungoli/amazonproductsearch-minidataset-input-embeddings


https://www.kaggle.com/code/abhishekmungoli/two-tower-retrieval-recommendation-model-training:

In [ ]:
!pip install chromadb
!pip install sentence_transformers
!pip install tfidf_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.0 MB/s et

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import os
import warnings
import chromadb
from sentence_transformers import SentenceTransformer
import shutil

ModuleNotFoundError: No module named 'chromadb'

In [1]:
# Load the saved sample_2c DataFrame from the CSV file
df_sample_2c = pd.read_csv('/content/drive/MyDrive/Grainger_Ashrith/Final/sample_2c_full_data.csv')

df_sample_2c.head()

NameError: name 'pd' is not defined

In [ ]:
# 2a. Create the Product Corpus (de-duplicated products)
print("Processing data into a unique Product Corpus...")
product_columns = [
    'product_id',
    'product_title',
    'product_description',
    'product_bullet_point',
    'product_brand',
    'product_color'
]
product_corpus_df = df_sample_2c[product_columns].drop_duplicates(subset=['product_id']).reset_index(drop=True)

# Fill NaNs with empty strings
text_cols_to_fill = product_columns[1:] # All except product_id
for col in text_cols_to_fill:
    product_corpus_df[col] = product_corpus_df[col].fillna('')

# Combine all text fields into a single 'product_text' for embedding
product_corpus_df['product_text'] = (
    product_corpus_df['product_title'] + ' ' +
    product_corpus_df['product_brand'] + ' ' +
    product_corpus_df['product_color'] + ' ' +
    product_corpus_df['product_description'] + ' ' +
    product_corpus_df['product_bullet_point']
)
# Clean up extra whitespace
product_corpus_df['product_text'] = product_corpus_df['product_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

print(f"Created a corpus of {len(product_corpus_df)} unique products.")

# 2b. Create the Query Evaluation Set (query-to-product pairs)
query_eval_set = df_sample_2c[['query', 'query_id', 'product_id', 'esci_label']].copy()
print(f"Created an evaluation set of {len(query_eval_set)} query-product pairs.")

Processing data into a unique Product Corpus...
Created a corpus of 485 unique products.
Created an evaluation set of 485 query-product pairs.


## 3a: tf-idf

In [ ]:
# --- Step 3: Create TF-IDF (Sparse) Vector Index ---
documents = product_corpus_df['product_text'].tolist()

print(f"Creating TF-IDF embeddings (sparse vectors) for {len(documents)} documents...")
# Initialize the TF-IDF Vectorizer
# Using stop words and limit to the top 5000 most frequent terms
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Create the sparse TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(f"TF-IDF matrix created with shape: {tfidf_matrix.shape}")

# Create the in-memory index using scikit-learn
# Using 'cosine' similarity and 'brute' force, which is efficient for sparse matrices
print("Building in-memory sparse index with scikit-learn (NearestNeighbors)...")
n_neighbors = 10
nn_index = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute')
nn_index.fit(tfidf_matrix)
print("In-memory sparse index built successfully.")


Creating TF-IDF embeddings (sparse vectors) for 485 documents...
TF-IDF matrix created with shape: (485, 5000)
Building in-memory sparse index with scikit-learn (NearestNeighbors)...
In-memory sparse index built successfully.


In [ ]:
test_query = "6 dining chairs"

# 2. Find the ground truth for this query from our in-memory DataFrame
ground_truth_df = query_eval_set[query_eval_set['query'] == test_query]
ground_truth_ids = ground_truth_df['product_id'].tolist()

print(f"Test Query: '{test_query}'")
print(f"Ground Truth 'Exact' Product IDs ({len(ground_truth_ids)}): {ground_truth_ids}")

# 3. Embed the test query *using the same vectorizer*
# .transform returns a sparse 2D matrix
query_vector = tfidf_vectorizer.transform([test_query])

# 4. Search the index
print(f"\nSearching index for Top {n_neighbors} results...")
# .kneighbors returns (distances, indices)
distances, indices = nn_index.kneighbors(query_vector)

# Flatten the results from 2D to 1D
result_indices = indices[0]
result_distances = distances[0]


Test Query: '6 dining chairs'
Ground Truth 'Exact' Product IDs (22): ['B08CZ6TC2L', 'B08HQG1MFS', 'B08K2K3J4C', 'B08K2V66N8', 'B08K8VDTW8', 'B086X1BK5G', 'B098D24ZZB', 'B098J9KBV3', 'B09BYVLGYG', 'B09CGQRYNK', 'B084DX126T', 'B08JJ4HWWB', 'B00STRXIOG', 'B01GOM6OUM', 'B071QXXTV6', 'B078WZLFHG', 'B07DRF78SN', 'B07DK2JK2T', 'B07TZNFWCL', 'B07TRM7P5D', 'B07WMQ1V7V', 'B07HH3K4SK']

Searching index for Top 10 results...


In [ ]:
# 5. Display results
print("Search Results (lower distance is better):")

# Get the full product info from the original corpus_df using the indices
results_df = product_corpus_df.iloc[result_indices].copy()

# Add the distance and a ground_truth check
results_df['_distance'] = result_distances
results_df['is_ground_truth'] = results_df['product_id'].isin(ground_truth_ids)

# Display the relevant columns
print(results_df[['product_id', '_distance', 'is_ground_truth', 'product_title']])

# Calculate a simple metric for this query
matches_in_top_10 = results_df['is_ground_truth'].sum()
print(f"\nQuery-specific Metric: Found {matches_in_top_10} out of {len(ground_truth_ids)} ground truth items in the Top 10 results.")


Search Results (lower distance is better):
    product_id  _distance  is_ground_truth  \
1   B08HQG1MFS   0.393061             True   
4   B08K8VDTW8   0.397773             True   
20  B07WMQ1V7V   0.416182             True   
19  B07TRM7P5D   0.432007             True   
14  B071QXXTV6   0.452820             True   
8   B09BYVLGYG   0.461786             True   
16  B07DRF78SN   0.489932             True   
3   B08K2V66N8   0.492087             True   
6   B098D24ZZB   0.506539             True   
5   B086X1BK5G   0.512308             True   

                                        product_title  
1   CozyCasa Dining Chairs Modern Style Dining Cha...  
4   Modern Dining Chairs Set of 6 - Faux Leather D...  
20  Homy Grigio Aristocratic Style Dining Chair No...  
19  ZENY Leather Dining Chairs with Wood Legs Chai...  
14  HomePop Parsons Classic Upholstered Accent Din...  
8   Guyou Dining Chair Set of 6 - Velvet Upholster...  
16  IDS Home Dining Room Chair for Kitchen, Mid Ce...  
3 

## 3b: Dense model (all-MiniLM-L6-v2)




In [ ]:
# Step 1: Setup, Constants, and Model Loading
MODEL_NAME = 'all-MiniLM-L6-v2'
DB_PATH = "./chroma_data"
COLLECTION_NAME = "product_embeddings"

model = SentenceTransformer(MODEL_NAME)
print("Model loaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully.


In [ ]:
# Step 3: Embed Documents for Indexing ---
documents = product_corpus_df['product_text'].tolist()
product_ids = product_corpus_df['product_id'].tolist()

print(f"Embedding {len(documents)}")
embeddings = model.encode(documents, show_progress_bar=True)
print("Embeddings generated.")

# Prepare data for ChromaDB
str_product_ids = [str(pid) for pid in product_ids]
metadatas = [{"product_id": pid, "text": doc} for pid, doc in zip(product_ids, documents)]

# --- Step 4: Create the ChromaDB Persistent Index ---
# Clean up old data directory if it exists, for a clean run
if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH)

# Initialize a persistent ChromaDB client
client = chromadb.PersistentClient(path=DB_PATH)

# Create a collection
print(f"Creating ChromaDB collection '{COLLECTION_NAME}' at '{DB_PATH}'...")
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={"hnsw:space": "l2"} # 'l2' (Euclidean) is standard for all-MiniLM
)

# Add the data to the collection
print(f"Adding {len(str_product_ids)} vectors to collection...")
collection.add(
    embeddings=embeddings.tolist(),
    documents=documents,
    metadatas=metadatas,
    ids=str_product_ids # Chroma requires a list of unique string IDs
)

print(f"Successfully created collection with {collection.count()} vectors.")


Embedding 485


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Embeddings generated.
Creating ChromaDB collection 'product_embeddings' at './chroma_data'...
Adding 485 vectors to collection...


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
# Step 5: Run a Test Query

test_query = "6 dining chairs"

# 2. Find the ground truth for this query from our in-memory DataFrame
ground_truth_df = query_eval_set[query_eval_set['query'] == test_query]
ground_truth_ids = ground_truth_df['product_id'].tolist()

print(f"Test Query: '{test_query}'")
print(f"Ground Truth 'Exact' Product IDs ({len(ground_truth_ids)}): {ground_truth_ids}")

# 3. Embed the test query
query_vector = model.encode([test_query]).tolist()

# 4. Search the index
n_neighbors = 10
print(f"\nSearching index for Top {n_neighbors} results...")
search_results = collection.query(
    query_embeddings=query_vector,
    n_results=n_neighbors,
)

# 5. Display results
print("Search Results (lower distance is better):")

# Process Chroma's output format
result_metadatas = search_results['metadatas'][0]
result_distances = search_results['distances'][0]

# Create a DataFrame for easy viewing
results_df = pd.DataFrame({
    'product_id': [meta['product_id'] for meta in result_metadatas],
    '_distance': result_distances
})

# Add a column to show if the result is a "Ground Truth" match
results_df['is_ground_truth'] = results_df['product_id'].isin(ground_truth_ids)

# Join with corpus_df to get the title
results_df = results_df.merge(product_corpus_df[['product_id', 'product_title']], on='product_id', how='left')

# Print the relevant columns
print(results_df[['product_id', '_distance', 'is_ground_truth', 'product_title']])

# Calculate a simple metric for this query
matches_in_top_10 = results_df['is_ground_truth'].sum()
print(f"\nQuery-specific Metric: Found {matches_in_top_10} out of {len(ground_truth_ids)} ground truth items in the Top 10 results.")

print("\n--- Compare this to our TF-IDF result ---")
print("TF-IDF Metric: Found 10 out of 22 ground truth items in the Top 10 results.")

Test Query: '6 dining chairs'
Ground Truth 'Exact' Product IDs (22): ['B08CZ6TC2L', 'B08HQG1MFS', 'B08K2K3J4C', 'B08K2V66N8', 'B08K8VDTW8', 'B086X1BK5G', 'B098D24ZZB', 'B098J9KBV3', 'B09BYVLGYG', 'B09CGQRYNK', 'B084DX126T', 'B08JJ4HWWB', 'B00STRXIOG', 'B01GOM6OUM', 'B071QXXTV6', 'B078WZLFHG', 'B07DRF78SN', 'B07DK2JK2T', 'B07TZNFWCL', 'B07TRM7P5D', 'B07WMQ1V7V', 'B07HH3K4SK']

Searching index for Top 10 results...
Search Results (lower distance is better):
   product_id  _distance  is_ground_truth  \
0  B07TZNFWCL   0.538012             True   
1  B08K8VDTW8   0.708939             True   
2  B084DX126T   0.722068             True   
3  B08JJ4HWWB   0.740451             True   
4  B078WZLFHG   0.789627             True   
5  B077GVYGT6   0.812472            False   
6  B09BYVLGYG   0.812662             True   
7  B098D24ZZB   0.844661             True   
8  B071QXXTV6   0.853706             True   
9  B07DK2JK2T   0.876175             True   

                                       produ

## Evalvate on

In [ ]:
# --- Add this import to your import cell, or run this in a new cell ---
from tqdm import tqdm # For a nice progress bar

# ---
# Part 4: Define Evaluation Metrics (New Cell)
# ---
print("\n--- Part 4: Defining Evaluation Metrics ---")

def calculate_reciprocal_rank(retrieved_ids, ground_truth_ids):
    """Calculates the reciprocal rank for a single query."""
    # Create a set for fast lookup
    ground_truth_set = set(ground_truth_ids)
    for i, p_id in enumerate(retrieved_ids):
        if p_id in ground_truth_set:
            return 1.0 / (i + 1) # Rank is i+1
    return 0.0

def calculate_recall_at_k(retrieved_ids, ground_truth_ids, k):
    """Calculates HITS@k (Recall@k) for a single query."""
    ground_truth_set = set(ground_truth_ids)
    retrieved_at_k = set(retrieved_ids[:k])

    hits = len(ground_truth_set.intersection(retrieved_at_k))

    # Handle the case where there are no ground truth IDs (shouldn't happen, but good practice)
    if not ground_truth_set:
        return 0.0

    return hits / len(ground_truth_set)

print("Metric functions (MRR, HITS@k) defined.")



--- Part 4: Defining Evaluation Metrics ---
Metric functions (MRR, HITS@k) defined.


In [ ]:
N_NEIGHBORS = 10 # This should match the N you used for your indexes

# Create a ground truth map for easy lookup: {query: [list_of_product_ids]}
# We group by 'query' text, not 'query_id', as it's more direct
ground_truth_map = query_eval_set.groupby('query')['product_id'].apply(list).to_dict()
unique_queries = list(ground_truth_map.keys())
print(f"Found {len(unique_queries)} unique queries to evaluate.")

# Lists to store scores for each query
tfidf_mrrs = []
tfidf_recalls_at_1 = []
tfidf_recalls_at_5 = []
tfidf_recalls_at_10 = []

dense_mrrs = []
dense_recalls_at_1 = []
dense_recalls_at_5 = []
dense_recalls_at_10 = []

# Loop through all unique queries with a progress bar
for query in tqdm(unique_queries, desc="Evaluating Queries"):
    ground_truth_ids = ground_truth_map[query]

    # --- 1. Evaluate TF-IDF ---
    query_vector_tfidf = tfidf_vectorizer.transform([query])
    # *** FIX: Use 'nn_index' as defined in your '3a' cell ***
    distances, indices = nn_index.kneighbors(query_vector_tfidf)

    # Get the product_ids from the indices
    retrieved_ids_tfidf = product_corpus_df.iloc[indices[0]]['product_id'].tolist()

    # Calculate and store scores
    tfidf_mrrs.append(calculate_reciprocal_rank(retrieved_ids_tfidf, ground_truth_ids))
    tfidf_recalls_at_1.append(calculate_recall_at_k(retrieved_ids_tfidf, ground_truth_ids, k=1))
    tfidf_recalls_at_5.append(calculate_recall_at_k(retrieved_ids_tfidf, ground_truth_ids, k=5))
    tfidf_recalls_at_10.append(calculate_recall_at_k(retrieved_ids_tfidf, ground_truth_ids, k=N_NEIGHBORS))

    # --- 2. Evaluate Dense Model (ChromaDB) ---
    query_vector_dense = model.encode([query]).tolist()
    # *** FIX: Use 'collection' as defined in your '3b' cell ***
    search_results = collection.query(
        query_embeddings=query_vector_dense,
        n_results=N_NEIGHBORS,
    )

    # Get the product_ids from the metadata
    retrieved_metadatas = search_results['metadatas'][0]
    retrieved_ids_dense = [meta['product_id'] for meta in retrieved_metadatas]

    # Calculate and store scores
    dense_mrrs.append(calculate_reciprocal_rank(retrieved_ids_dense, ground_truth_ids))
    dense_recalls_at_1.append(calculate_recall_at_k(retrieved_ids_dense, ground_truth_ids, k=1))
    dense_recalls_at_5.append(calculate_recall_at_k(retrieved_ids_dense, ground_truth_ids, k=5))
    dense_recalls_at_10.append(calculate_recall_at_k(retrieved_ids_dense, ground_truth_ids, k=N_NEIGHBORS))

print("Evaluation complete.")


Found 50 unique queries to evaluate.


Evaluating Queries:  82%|████████▏ | 41/50 [00:01<00:00, 30.79it/s]


InternalError: Error executing plan: Internal error: error returned from database: (code: 14) unable to open database file

In [ ]:

# ---
# Part 6: Display Final Quantified Metrics (New Cell)
# ---
print("\n--- FINAL QUANTIFIED METRICS (Averaged over all queries) ---")

print("\n--- TF-IDF (Keyword Baseline) ---")
print(f"  Mean Reciprocal Rank (MRR): {np.mean(tfidf_mrrs):.4f}")
print(f"  HITS@1 (Recall@1):          {np.mean(tfidf_recalls_at_1):.4f}")
print(f"  HITS@5 (Recall@5):          {np.mean(tfidf_recalls_at_5):.4f}")
print(f"  HITS@10 (Recall@10):        {np.mean(tfidf_recalls_at_10):.4f}")

print("\n--- Dense Model (all-MiniLM-L6-v2) ---")
print(f"  Mean Reciprocal Rank (MRR): {np.mean(dense_mrrs):.4f}")
print(f"  HITS@1 (Recall@1):          {np.mean(dense_recalls_at_1):.4f}")
print(f"  HITS@5 (Recall@5):          {np.mean(dense_recalls_at_5):.4f}")
print(f"  HITS@10 (Recall@10):        {np.mean(dense_recalls_at_10):.4f}")

print("\n--- Analysis ---")
print("MRR measures the average rank of the *first* correct item. Higher is better.")
print("HITS@N measures what percentage of correct items were found in the Top N. Higher is better.")


# Step 4 - A different approach (Two tower)

## Creating the training data

In [1]:
# ---
# --- CELL 1: Data Preparation for Fine-Tuning ---
# ---
# This script loads your FULL dataset, removes the 50 queries
# from our test set, and builds 30,000 "triplet" examples
# of (query, positive_product, negative_product)
# ---

import pandas as pd
from sentence_transformers import InputExample
from tqdm.autonotebook import tqdm
import warnings
import random

warnings.filterwarnings("ignore")

print("--- Iteration 3: Data Preparation ---")

# --- 1. Load All Data (as per your snippet) ---
try:
    # Load the raw examples and products files
    df_examples = pd.read_parquet('/content/drive/MyDrive/Grainger_Ashrith/shopping_queries_dataset_examples.parquet')
    df_products = pd.read_parquet('/content/drive/MyDrive/Grainger_Ashrith/shopping_queries_dataset_products.parquet')

    # Load our 50-query TEST set (to exclude from training)
    df_sample_2c = pd.read_csv('/content/drive/MyDrive/Grainger_Ashrith/Final/sample_2c_full_data.csv')

    print(f"Loaded {len(df_examples)} examples and {len(df_products)} products.")

except FileNotFoundError:
    print("Error: Could not find the Parquet or CSV files.")
    print("Please ensure all source files are in your Drive.")
    # This cell will fail here if files are not found, which is intended.

# --- 2. Merge and Filter Data ---
print("Merging examples and products...")
df_merged = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale', 'product_id'],
    right_on=['product_locale', 'product_id']
)

# Filter for Task 1 (using the 'small_version' flag)
df_task_1 = df_merged[df_merged["small_version"] == 1].copy()
print(f"Filtered to {len(df_task_1)} rows for Task 1.")


# --- 3. Get Test Queries to Exclude ---
# This is CRITICAL to prevent data leakage.
test_queries = set(df_sample_2c['query'].unique())
print(f"Identified {len(test_queries)} unique queries to exclude from our new training set.")


# --- 4. Create Product Text and Mapping ---
print("Creating product text map...")
product_cols = ['product_title', 'product_brand', 'product_color', 'product_description', 'product_bullet_point']
for col in product_cols:
    df_products[col] = df_products[col].fillna('')

df_products['product_text'] = (
    df_products['product_title'] + ' ' +
    df_products['product_brand'] + ' ' +
    df_products['product_color'] + ' ' +
    df_products['product_description'] + ' ' +
    df_products['product_bullet_point']
)
df_products['product_text'] = df_products['product_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Create a fast lookup map for product_id -> product_text
# We use df_products here as it's the complete, unique product catalog
product_map = dict(zip(df_products['product_id'], df_products['product_text']))
print("Product text map created.")


# --- 5. Build "Triplet" Training Set ---
print("Building (query, positive, negative) training triplets...")

# Use df_task_1 as our source data
df_train_source = df_task_1[df_task_1['product_locale'] == 'us'].copy()

# *** CRITICAL STEP: Filter out the test queries ***
df_train_source = df_train_source[~df_train_source['query'].isin(test_queries)]
print(f"Filtered to {len(df_train_source)} rows (US locale, training queries only).")

# Get positive (E) and negative (I) pairs
positives = df_train_source[df_train_source['esci_label'] == 'E'][['query', 'product_id']]
negatives = df_train_source[df_train_source['esci_label'] == 'I'][['query', 'product_id']]

# Create a DataFrame of (query, product_id_pos, product_id_neg)
# This finds queries that have *both* E and I labels
df_triplets = pd.merge(
    positives,
    negatives,
    on='query',
    suffixes=('_pos', '_neg')
)
print(f"Found {len(df_triplets)} potential (query, pos, neg) triplets.")

# --- 6. Sample and Format for Training ---
N_TRAINING_EXAMPLES = 30000

if len(df_triplets) == 0:
    print("\n--- ERROR ---")
    print("No (query, positive, negative) triplets were found.")
    print("This can happen if your training data has no queries with both 'E' and 'I' labels.")
elif len(df_triplets) > N_TRAINING_EXAMPLES:
    print(f"Sampling down to {N_TRAINING_EXAMPLES} random triplets...")
    df_triplets_sample = df_triplets.sample(N_TRAINING_EXAMPLES, random_state=42)
else:
    print(f"Using all {len(df_triplets)} available triplets for training.")
    df_triplets_sample = df_triplets

# Create the final list of InputExample objects
train_examples = []
for _, row in tqdm(df_triplets_sample.iterrows(), total=len(df_triplets_sample), desc="Formatting Triplets"):
    query = row['query']

    # Use .get() for safety, in case a product_id is missing from the map
    pos_text = product_map.get(row['product_id_pos'], '')
    neg_text = product_map.get(row['product_id_neg'], '')

    # Skip if any text is missing
    if not query or not pos_text or not neg_text:
        continue

    train_examples.append(InputExample(texts=[query, pos_text, neg_text]))

print(f"\nSuccessfully created {len(train_examples)} training examples.")
print("The variable 'train_examples' is now ready for the next cell.")


--- Iteration 3: Data Preparation ---
Loaded 2621288 examples and 1814924 products.
Merging examples and products...
Filtered to 1118011 rows for Task 1.
Identified 50 unique queries to exclude from our new training set.
Creating product text map...
Product text map created.
Building (query, positive, negative) training triplets...
Filtered to 600342 rows (US locale, training queries only).
Found 819755 potential (query, pos, neg) triplets.
Sampling down to 30000 random triplets...


Formatting Triplets:   0%|          | 0/30000 [00:00<?, ?it/s]


Successfully created 30000 training examples.
The variable 'train_examples' is now ready for the next cell.


In [2]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

# --- 1. Check for Training Data ---
if 'train_examples' not in locals() or not train_examples:
    print("Error: 'train_examples' not found or is empty.")
    print("Please run the 'Data Preparation' cell (Cell 1) first.")
else:
    print("--- Iteration 3: Fine-Tuning ---")

    # --- 2. Setup Model and Training ---

    # Define model name
    MODEL_NAME = 'all-MiniLM-L6-v2'

    FINETUNED_MODEL_PATH = './my-finetuned-model'

    print(f"Loading base model: {MODEL_NAME}")
    # Load the base model
    model = SentenceTransformer(MODEL_NAME)

    # --- Setup Training ---
    train_batch_size = 16
    num_epochs = 1

    # Create a DataLoader for our triplet examples
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

    # Define the loss function: TripletLoss
    # This will try to pull (anchor, positive) closer
    # and push (anchor, negative) further apart.
    train_loss = losses.TripletLoss(model=model)

    # --- 3. Start Fine-Tuning ---

    warmup_steps = int(len(train_dataloader) * 0.1) # 10% of steps for warmup

    print(f"Starting fine-tuning for {num_epochs} epoch...")

    # Call model.fit() to train
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=num_epochs,
        warmup_steps=warmup_steps,
        output_path=FINETUNED_MODEL_PATH,
        show_progress_bar=True
        # (callback argument removed)
    )


    print("\n--- Fine-tuning complete! ---")
    print(f"New fine-tuned model saved to: {FINETUNED_MODEL_PATH}")
    print("\nYou can now proceed to the final evaluation.")


--- Iteration 3: Fine-Tuning ---
Loading base model: all-MiniLM-L6-v2
Starting fine-tuning for 1 epoch...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 